In [13]:
using CSV
using DataFrames
using Serialization
using HDF5

In [14]:
df = DataFrame(CSV.File("top20_adjacency_mat.csv"));
population_sizes = Dict(
    "PEK" => 16_938_000,
    "LHR" => 9_800_000,
    "CGK" => 9_733_000,
    "DEL" => 16_787_941,
    "BOM" => 12_442_373,
    "ATL" => 4_661_000,
    "HND" => 36_975_000,
    "BKK" => 8_484_000,
    "CDG" => 10_514_000,
    "CAN" => 10_546_000,
    "PVG" => 20_948_000,
    "SIN" => 5_176_000,
    "LAX" => 12_197_000,
    "KUL" => 6_005_000,
    "CTU" => 7_570_000,
    "AMS" => 1_073_000,
    "GRU" => 11_310_000,
    "DEN" => 2_428_000,
    "HKG" => 7_066_000,
    "MNL" => 12_075_689
);


In [15]:
df[df.Origin .== "AMS",:]

Row,Origin,AMS,ATL,BKK,BOM,CAN,CDG,CGK,CTU,DEL,DEN,GRU,HKG,HND,KUL,LAX,LHR,MNL,PEK,PVG,SIN
,String3,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,AMS,0.0,1.49482e5,1.9045e5,89757.0,2552.79,1.79445e5,1161.55,42272.0,91847.4,1315.21,47736.9,2.02192e5,131.195,91241.8,101692.0,2.77285e5,73382.7,206660.0,170692.0,1.44559e5


In [16]:
# this adjacency matrix will store the number of migrations per origin population in a given year
new_adjacency_matrix = Array{Float64}(undef,size(df,1), size(df,2)-1)
population_sizes_vector = Vector{Int64}(undef,length(population_sizes))

for from_idx = 1:size(new_adjacency_matrix,2)
    for to_idx = 1:size(new_adjacency_matrix,1)
        new_adjacency_matrix[to_idx,from_idx] = df[from_idx,to_idx+1] / population_sizes[df[from_idx,1]]
    end
    # record the population size for the corresponding col of the migration matrix
    population_sizes_vector[from_idx] = population_sizes[df[from_idx,1]] 
end

# Currently, this is the rate per year, we want the rate in units of the recovery time (typically 7 days, 1 week) so divide by 52
new_adjacency_matrix ./= 52


dfnew = DataFrame(new_adjacency_matrix,:auto)
dfnew[!, :population_sizes] = population_sizes_vector  # Append as a new column

CSV.write("cleaned_adjacency_matrix.csv", dfnew)
# serialize("cleaned_adjacency_matrix", (new_adjacency_matrix, population_sizes_vector))
# h5write("cleaned_adjacency_matrix.h5", "matrix", new_adjacency_matrix)
# h5write("cleaned_adjacency_matrix.h5", "populations", population_sizes_vector)

"cleaned_adjacency_matrix.csv"

In [20]:
df = CSV.read("cleaned_adjacency_matrix.csv", DataFrame)
new_adjacency_matrix2 = Matrix(df[:, 1:end-1])  # Assuming the last column is the population sizes vector
# population_sizes_vector = df[:,end]

for i = 1:20
    println(i, new_adjacency_matrix2[i,i])
end

10.0
20.0
30.0
40.0
50.0
60.0
70.0
80.0
90.0
100.0
110.0
120.0
130.0
140.0
150.0
160.0
170.0
180.0
190.0
200.0


In [11]:
a = randn(3,3)

3×3 Matrix{Float64}:
 -0.249819  0.298761  -0.191842
  0.239972  2.12436   -1.13304
 -1.75094   2.25426   -1.0761

In [12]:
b = similar(a)

3×3 Matrix{Float64}:
 5.0e-324      1.50427e-311  1.50425e-311
 5.0e-324      1.0e-323      1.50425e-311
 1.50425e-311  1.0e-323      1.50237e-311

In [23]:
migration_matrix , popsiz= deserialize("cleaned_adjacency_matrix")

([0.0 0.0005806082356047729 … 0.00015669937280592233 0.0005540071350760907; 0.002679083764248333 0.0 … 2.7439891290337976e-6 1.1827096922185234e-5; … ; 0.003059215714388128 1.1118315529434092e-5 … 0.0 0.0025055561532145404; 0.002590854407125959 1.0984055958609081e-5 … 0.0005722101866802045 0.0], [1073000, 4661000, 8484000, 12442373, 10546000, 10514000, 9733000, 7570000, 16787941, 2428000, 11310000, 7066000, 36975000, 6005000, 12197000, 9800000, 12075689, 16938000, 20948000, 5176000])

In [24]:
popsiz

20-element Vector{Int64}:
  1073000
  4661000
  8484000
 12442373
 10546000
 10514000
  9733000
  7570000
 16787941
  2428000
 11310000
  7066000
 36975000
  6005000
 12197000
  9800000
 12075689
 16938000
 20948000
  5176000